# feature 따른 예측값 변화_PDP, ICE



## 1.환경준비

### 1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore')

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.backend import clear_session

### 2) 데이터 로딩

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/boston.csv')
data.head()

* 변수 설명
    * crim : 범죄율
    * zn : 대저택 비율
    * indus : 상업지역 비율
    * chas : 찰스 강변 여부
    * nox : 일산화질소 농도(공기오염도)
    * rm : 평균 주택당 방 수
    * age : 30년 이상된 주택 비율
    * dis : 주요 업무 지역 접근성 지수
    * rad : 고속도로 접근성 지수
    * tax  1만 달러당 재산세
    * ptratio : 교사1명당 학생수
    * lstat : 하위계층 비율
    * mdev : 타운별 집값 중위수(단위 : 1000달러)

### 3) 필요한 함수 생성

In [ ]:
# 변수 중요도 plot
def plot_feature_importance(importance, names, topn = 'all'):
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_temp = pd.DataFrame(data)

    fi_temp.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_temp.reset_index(drop=True, inplace = True)

    if topn == 'all' :
        fi_df = fi_temp.copy()
    else :
        fi_df = fi_temp.iloc[:topn]

    plt.figure(figsize=(10,8))
    sns.barplot(x='feature_importance', y='feature_names', data = fi_df)

    plt.xlabel('importance')
    plt.ylabel('feature names')
    plt.grid()

    return fi_df

## 2.데이터 준비


### 1) x, y로 나누기

In [ ]:
target = 'medv'
x = data.drop(target, axis = 1)
y = data.loc[:,target]

### 2) 가변수화

### 3) train : validation 분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .2, random_state = 20)
x_train.shape

## 3.모델생성

* RandomForest 모델을 만들고 개별 데이터 분석 단위에 대해서 해석해 보겠습니다.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = RandomForestRegressor()
model.fit(x_train, y_train)

## 4.PDP, ICE 원리 이해

### 1) 특정 행에서, 각 변수의 영향력 살펴보기

* 학습데이터의 첫 행에 대해서 각 변수의 영향력을 살펴보려고 합니다.

In [ ]:
data1 = x_train.iloc[[0]]
data1

* 변수 중 crim의 영향력을 살펴보겠습니다.
    * ① x_train에서 crim의 값들을 뽑습니다.
    * ② data1의 crim 에 ①의 값을 하나씩 넣으며 예측값을 계산합니다.
    * ③ 이를 그래프로 그립니다. x축 : crim, y축 : 예측값

In [ ]:
# ① x_train에서 crim의 값들을 뽑습니다.
crim_values = x_train['crim'].sort_values()
crim_values

In [ ]:
# ② data1의 crim 에 ①의 값을 하나씩 넣으며 예측값을 계산합니다.
pred_crim = []

for cr in crim_values :
    data1['crim'] = cr
    pred_crim.append(model.predict(data1)[0])

pred_crim

In [ ]:
# ③ 이를 그래프로 그립니다. x축 : crim, y축 : 예측값
sns.lineplot(x = crim_values, y = pred_crim)
plt.ylim(y_train.min(), y_train.max())  # 실제 값의 범위 지정
plt.grid()
plt.show()

In [ ]:
y_train.describe()

* 다른 변수의 영향력도 살펴봅시다.
    * lstat
    * ptratio
    * rm

* 반복되는 코드이므로 함수로 생성해 봅시다.

In [ ]:
def ice_plot(model, x, y, data_1row, var) :

    # ① x_train에서 crim의 값들을 뽑습니다.
    x_values = x[var].sort_values()

    # ② data1의 crim 에 ①의 값을 하나씩 넣으며 예측값을 계산합니다.
    pred = []

    for v in x_values :
        data_1row[var] = v
        pred.append(model.predict(data_1row)[0])

    # ③ 이를 그래프로 그립니다. x축 : crim, y축 : 예측값
    sns.lineplot(x = x_values, y = pred)
    plt.ylim(y.min(), y.max())  # 실제 값의 범위 지정
    plt.grid()
    plt.show()

In [ ]:
data1.shape, x_train.shape

In [ ]:
ice_plot(model, x_train, y_train, data1, 'lstat')

In [ ]:
ice_plot(model, x_train, y_train, data1, 'ptratio')

In [ ]:
ice_plot(model, x_train, y_train, data1, 'rm')

### 2) 여러행에서, 특정 변수의 영향력 살펴보기

* ① x_train에서 20개의 행을 랜덤으로 뽑습니다.
* ② 20개 행에 대해서 lstat의 영향력을 그래프로 그립니다.(ice plot)
* ③ 이들의 평균선을 추가합니다.(PDP!)

In [ ]:
# ① x_train에서 20개의 행을 랜덤으로 뽑습니다.
data20 = x_train.sample(20, random_state = 2023)
data20

In [ ]:
# ② 20개 행에 대해서 lstat의 영향력을 그래프로 그립니다.(ice plot)

var = 'lstat'

for i in range(20) :

    # 특정 변수의 값을 뽑아 정렬
    x_values = data20[var].sort_values()

    # 행 하나씩 추출
    data1 = data20.iloc[[i]]

    # 추출된 한 행에서, 특정 변수의 값을 하나씩 대입하고 예측
    pred = []
    for v in x_values :
        data1[var] = v
        pred.append(model.predict(data1)[0])

    # 한 행에 대한 라인플롯, x축 : 특정 변수의 값, y축 : 예측값.
    sns.lineplot(x = x_values, y = pred, color = 'gray', linewidth = 0.3)
    plt.ylim(y_train.min(), y_train.max())  # 실제 값의 범위 지정

plt.grid()
plt.show()

In [ ]:
# ③ 이들의 평균선을 추가합니다.(PDP!)

# 개별 라인차트들 -----------------------------------------------------
var = 'lstat'
pred_values = []
for i in range(20) :

    x_values = data20[var].sort_values()
    data1 = data20.iloc[[i]]
    pred = []

    for v in x_values :
        data1[var] = v
        pred.append(model.predict(data1)[0])

    pred_values.append(pred)

    sns.lineplot(x = x_values, y = pred, color = 'gray', linewidth = 0.3)
    plt.ylim(y_train.min(), y_train.max())  # 실제 값의 범위 지정

# -----------------------------------------------------------------------
# 평균선 추가 -----------------------------------------------------------
pred_mean = np.array(pred_values).mean(axis = 0)
sns.lineplot(x = x_values, y = pred_mean, color = 'red', linewidth = 1, label = 'average')

plt.grid()
plt.show()

## 5.함수 이용하기

In [ ]:
from sklearn.inspection import PartialDependenceDisplay, partial_dependence

### 1) PDP 그리기
* model : 이미 만든 모델
* features : 분석할 대상 feature
* X : 데이터셋(x)
* kind : 'both' ➡ plotting both Partial dependence (PD) and individual conditional expectation (ICE)

In [ ]:
var = 'lstat'
PartialDependenceDisplay.from_estimator(model, data20, [var], kind="both")
plt.grid()
plt.show()

In [ ]:
var = 'rm'
temp = x_train.head(3).copy()

pdp = partial_dependence(model, features = [var], X = temp, kind = 'both')
print(pdp['average'])
print(pdp['individual'])
print(pdp['values'])

In [ ]:
PartialDependenceDisplay.from_estimator(model, temp, [var], kind="both")
plt.grid()
plt.show()

In [ ]:
temp

* 수동으로 만들기


In [ ]:
temp['rm'] = 6.172
temp
model.predict(temp)

* 전체 데이터에 대해서 살펴봅시다.

In [ ]:
PartialDependenceDisplay.from_estimator(model,
                                        x_train,
                                        ['rm'],
                                        kind="both")
plt.grid()
plt.show()

### 2) 두 변수의 영향력 비교
* 변수 입력 : 리스트로

In [ ]:
PartialDependenceDisplay.from_estimator(model, x_train, ['rm','lstat'])
plt.show()

### 3) 두 변수의 조합으로 분석
* 리스트 안에 튜플로 입력

In [ ]:
PartialDependenceDisplay.from_estimator(model, x_train, [('rm','lstat')])
plt.show()

## 6.실습 : 특정 feature 값의 변화에 따른 예측값 영향 분석

### 1) 데이터 준비

* 데이터 로딩

In [ ]:
# data data
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/Attrition2.csv"
data = pd.read_csv(path)
data.head(10)

|	변수 명	|	내용	|	구분	|
|	----	|	----	|	----	|
|	**Attrition**	|	이직여부, Yes = 1 , No = 0	|	**Target**	|
|	Age	|	나이	|	숫자	|
|	BusinessTravel	|	출장 빈도(범주)	|		|
|	Department	|	현 부서	|		|
|	DistanceFromHome	|	집-직장 거리(마일)	|	숫자	|
|	Education	|	교육수준(범주)	|	1 Below College, 2 College, 3 Bachelor, 4 Master, 5 Doctor	|
|	EducationField	|	전공	|		|
|	EnvironmentSatisfaction	|	근무환경에 대한 만족도(범주)	|	1 Low, 2 Good, 3 Excellent, 4 Outstanding	|
|	Gender	|	성별	|		|
|	JobInvolvement	|	직무 적극성(참여도)	|	1 Low, 2 Medium, 3 High, 4 Very High	|
|	JobRole	|	직무	|		|
|	JobSatisfaction	|	직무 만족도	|	1 Low, 2 Medium, 3 High, 4 Very High	|
|	MaritalStatus	|	결혼상태	|		|
|	MonthlyIncome	|	월급	|	숫자	|
|	NumCompaniesWorked	|	현재까지 근무한 회사 수	|	숫자	|
|	PercentSalaryHike	|	전년대비 급여인상율(%)	|	숫자	|
|	RelationshipSatisfaction	|	동료와의 관계 만족도	|	1 Low, 2 Medium, 3 High, 4 Very High	|
|	StockOptionLevel	|	스톡옵션 수준 0~3	|	범주	|
|	TotalWorkingYears	|	총 근무 연수	|	숫자	|
|	TrainingTimesLastYear	|	전년 교육훈련 횟수	|	숫자	|
|	WorkLifeBalance	|	워라밸. 일-삶 균형도	|	1 Bad, 2 Good, 3 Better, 4 Best	|
|	YearsAtCompany	|	현직장 근무 연수	|	숫자	|
|	YearsInCurrentRole	|	현직무 연수	|	숫자	|
|	YearsWithCurrManager	|	현 팀장과 근무한 연수	|	숫자	|


In [ ]:
target = 'Attrition'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

In [ ]:
y.value_counts(normalize = True)

* 가변수화 Dummy Variable

In [ ]:
dummy_vars = ['Education','Department','EducationField','Gender','JobRole','MaritalStatus']
x = pd.get_dummies(x, columns = dummy_vars, drop_first=True)

* 데이터 분할
    * 이미 test set은 분할되어 있다고 가정합니다.
    * 주어진 데이터를 train set : validation set 으로 분할

In [ ]:
# train_val에서 train : val = 8 : 2
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state = 20)

In [ ]:
print(x_train.shape, x_val.shape)

* class balance를 맞추기 위한 resampling

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote = SMOTE()
x_train, y_train = smote.fit_resample(x_train, y_train)

### 2) 모델링

* model : RandomForest, xgb, svm 등...

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier()
model.fit(x_train, y_train)

### 3) PDP

* model : 이미 만든 모델
* features : 아래 변수들에 대해 분석을 시도하시오.
* X : x_val
* kind
    * kind='average' results in the traditional PD plot(default)
    * kind='individual' results in the ICE plot
    * kind='both' results in plotting both the ICE and PD on the same plot.

In [ ]:
from sklearn.inspection import PartialDependenceDisplay, partial_dependence

#### ① 개별 변수 분석

* 중요한 feature 각각에 대해서 PDP를 그리고 해석해 보겠습니다.
    * kind 옵션 없이 그래프 그리기
    * kind = 'both'로 그래프 그리기

* feature 1 : 'MonthlyIncome'

* feature 2 : 'MaritalStatus_Married'

* feature 3 : 'DistanceFromHome'

* feature 4 : 'JobSatisfaction'

#### ② 두 변수와 예측값 비교
* 두 변수의 영향력 비교
* 두 변수의 조합으로 분석


* 조합1 : 'MonthlyIncome','MaritalStatus_Married'

* 조합2 : 'DistanceFromHome','JobSatisfaction'